In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Récupérer le code source
def scraper(page):
    ## Le lien de linkedin avec mot clé recherché "DATA"
    url='https://www.linkedin.com/jobs/search?keywords=Data&location=France&geoId=105015875&trk=public_jobs_jobs-search-bar_search-submit&original_referer=https%3A%2F%2Fwww.linkedin.com%2Fjobs%2Fsearch%3Fkeywords%3D%26location%3DFrance%26geoId%3D105015875%26f_TPR%3Dr2592000%26position%3D1%26pageNum%3D0&position=1&pageNum=0'    
    
    ## User-Agent permet de spécifier le navigateur qui va accèder a la page web. 
    ## Il envoie ces chaines de caractère 'Mozilla/5.0 (Windows NT 10.0...' qui ont pour objectif d'optimiser la page web en fonction de notre matériel et de vos logiciels.
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0'}
    
    ## requests permet de lire la page web
    r = requests.get(url,headers) 
    
    ## B4s permet de récupérer le contenu du code source de la page web, c'est-à-dire extraire des données de fichiers HTML et XML. 
    soup = BeautifulSoup(r.content, 'lxml')
    
    return soup

## Extraire les informations pertinentes sur les offres d'emploi
def extract(soup):
    ## On recherche dans le code source toutes les balises <div> ayant la class 'base-card.....'. cette balise + class contient les informations sur les offres d'emploi LinkendIn
    jobs = soup.find_all('div', class_='base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card' )
    
    for item in jobs:
        ## Extraire le titre de l'offre d'emploi à partir du code HTML en trouvant le premier élément <a> à l'intérieur de la balise <div> ayant la class 'base-card.....'. Puis supprime tout espace inutile "strip()"
        title = item.find('a').text.strip()
        
        ## Même lecture que le commentaire de title
        company = item.find('h4').text.strip()
        
        ## Accéder à l'attribut 'href' de l'élément <a>. L'attribut 'href' est utilisé pour spécifier l'URL de destination du lien.
        link = item.a['href']
        
        ## Même lecture que le commentaire de title
        location = item.find('span',class_='job-search-card__location').text.strip()
        
        ## Même lecture que le commentaire de title
        Date_posted = item.find('time', class_='job-search-card__listdate')
        
        ## Cette instruction vérifie si l'élément Date_posted existe dans la structure HTML.
        ## Si Date_posted existe, donc 
        if Date_posted is not None:
            ## Extrait le texte de Date_posted
            time = Date_posted.text.strip()
            ## Ensuite, stocker les résultats de l'offre d'emploi dans le dictionnaire
            job  = {
                'Title' : title,
                'Link' : link,
                'Company': company,
                'Location': location,
                'Job_posted':time }
            
            ## Une fois que le dictionnaire job est créé, il est ajouté à la liste "joblist"
            joblist.append(job)
    return
            
      
## créer une liste vide         
joblist =[]       

##
for i in range (0,300, 1):
    ## Exécution de la fonction scraper qui va Scaper les page de 0 a 299 (inclus).
    s = scraper(i)
    ## Ensuite, la fonction extract() est appelée avec l'objet soup retourné par scraper(). 
    ## Cette fonction extrait les détails des offres d'emploi de la page et les ajoute à la liste joblist.
    c = extract(s)

df = pd.DataFrame(joblist)
df.head()

,Title,Link,Company,Location,Job_posted
0,Data Analyst Junior (H/F),https://fr.linkedin.com/jobs/view/data-analyst...,Akademija Oxford,"Boulogne-Billancourt, Île-de-France, France",3 weeks ago
1,Data Analyst en alternance,https://fr.linkedin.com/jobs/view/data-analyst...,Akademija Oxford,"Ille-et-Vilaine, Brittany, France",4 days ago
2,Data Analyst Junior (H/F),https://fr.linkedin.com/jobs/view/data-analyst...,Akademija Oxford,"Boulogne-Billancourt, Île-de-France, France",2 weeks ago
3,Data analyst,https://fr.linkedin.com/jobs/view/data-analyst...,Akademija Oxford,"Boulogne-Billancourt, Île-de-France, France",2 weeks ago
4,Data Analyst,https://fr.linkedin.com/jobs/view/data-analyst...,Akademija Oxford,"Ille-et-Vilaine, Brittany, France",2 weeks ago


In [6]:
df.to_csv('Linkedin_job_list.csv')